In [1]:
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
tif_file = "E:/data/coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif"

In [7]:
def rgb2gray(rgb):
    r, g, b = rgb.read(1), rgb.read(2), rgb.read(3) #Index starts at 1 for image.read(index)
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def remove_channel(image):
    #Input : image.read()
    #Output : Image with the removed channels

    removed_channel = []
    for x in image[2:]: #Removes NDVI & AHN
        removed_channel.append(x)
    removed_channel = np.array(removed_channel)
    return removed_channel

In [45]:
src.read().shape[0]

6

In [41]:

src = rasterio.open(tif_file)

gray_im = rgb2gray(src)
gray_im = np.reshape(gray_im, (1, gray_im.shape[0], gray_im.shape[1]))

normalized_gray = NormalizeData(gray_im)*255

final_im = [0,1,2,3]
final_im[0] = normalized_gray.reshape(normalized_gray.shape[1],normalized_gray.shape[2]) #Greyscale
final_im[1] = src.read(4)    #Near Infra-red
final_im[2] = src.read(5)   #NDVI
final_im[3] = src.read(6)   #AHN

In [42]:
final_im = np.array(final_im)

In [43]:
file_name = "E:/data/coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_gray_scale.tif"
with rasterio.open(tif_file) as inds:
    meta = inds.profile.copy()
    #Number of bands
    meta["count"] = 4 
    with rasterio.open(file_name, "w", **meta) as outimg:
        outimg.write(final_im)